In [ ]:
pip install transformers xFormers diffusers pillow opencv-python numpy gradio torch==2.5.1

In [ ]:
pip install --upgrade torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
pip install --upgrade fastai


In [ ]:
# Higher model
import torch
from diffusers import StableDiffusionPipeline
import gradio as gr

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"

# Use Stable Diffusion 2.1 (More Compatible with Diffusers)
model_id = "stabilityai/stable-diffusion-2-1"

# Load the Stable Diffusion model
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

# Enable optimizations
pipe.enable_attention_slicing()
if device == "cuda":
    pipe.enable_xformers_memory_efficient_attention()
    pipe.unet = torch.compile(pipe.unet)

# Function to generate an image
def generate_image(prompt, steps, guidance):
    image = pipe(prompt, num_inference_steps=steps, guidance_scale=guidance).images[0]
    return image

# Launch Gradio UI with debug mode
gr.Interface(
    fn=generate_image,
    inputs=[
        gr.Textbox(label="Enter Prompt"),
        gr.Slider(10, 30, value=20, step=1, label="Inference Steps"),
        gr.Slider(1, 10, value=5, step=0.5, label="Guidance Scale")
    ],
    outputs="image"
).launch(share=True, debug=True)


In [ ]:
# test
import torch
from diffusers import StableDiffusionPipeline
import gradio as gr

model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

pipe.enable_attention_slicing()
if device == "cuda":
    pipe.enable_xformers_memory_efficient_attention()
    pipe.unet = torch.compile(pipe.unet)

def generate_image(prompt, steps=30, guidance=7.5):
    # Add negative prompt for better results
    negative_prompt = "ugly, blurry, bad quality, distorted, deformed, low resolution"

    # Generate image with fixed size and improved parameters
    image = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=steps,
        guidance_scale=guidance,
        width=1000,
        height=1000
    ).images[0]
    return image

gr.Interface(
    fn=generate_image,
    inputs=[
        gr.Textbox(label="Enter Prompt"),
        gr.Slider(20, 50, value=30, step=1, label="Inference Steps"),  # Increased default steps
        gr.Slider(1, 20, value=7.5, step=0.5, label="Guidance Scale")  # Wider range, better default
    ],
    outputs="image"
).launch(share=True, debug=True)

In [ ]:
# Main build
import torch
from diffusers import StableDiffusionPipeline
import gradio as gr

model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    safety_checker=None
).to(device)

pipe.enable_attention_slicing()
if device == "cuda":
    pipe.enable_xformers_memory_efficient_attention()
    pipe.enable_vae_slicing()
def generate_image(prompt, steps=30, guidance=7.5):
    # Enhanced negative prompt for better results
    negative_prompt = "ugly, blurry, bad quality, distorted, deformed, low resolution, worst quality, low quality, jpeg artifacts, watermark, text, signature, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, bad faces, broken arm and legs, deformed body, duplicated faces, duplicate, cloned objects, bad lighting, bad shading, bad colors, bad composition, bad contrast, bad saturation, bad hue, bad brightness, bad exposure, bad shadows, bad highlights, bad midtones, bad white balance, bad focus, bad framing, bad cropping, bad angle, bad perspective, bad lens, bad camera, bad camera settings, bad camera angle, bad camera lens, bad camera focus, bad camera exposure, bad camera framing, bad camera cropping, bad camera angle, bad camera perspective, bad camera white balance, bad camera settings, bad camera lighting, bad camera shadows, bad camera highlights, bad camera midtones, bad camera saturation, bad camera hue, bad camera brightness, bad camera contrast, bad camera colors, bad camera composition, bad camera shading, bad camera lighting, bad camera focus, bad camera framing, bad camera cropping, bad camera angle, bad camera perspective, bad camera lens, bad camera exposure, bad camera white balance, bad camera settings, bad camera angle, bad camera lens, bad camera focus, bad camera exposure, bad camera framing, bad camera cropping, bad camera angle, bad camera perspective, bad camera white balance, bad camera settings, bad camera lighting, bad camera shadows, bad camera highlights, bad camera midtones, bad camera saturation, bad camera hue, bad camera brightness, bad camera contrast, bad camera colors, bad camera composition, bad camera shading, bad camera lighting, bad camera focus, bad camera framing, bad camera cropping, bad camera angle, bad camera perspective, bad camera lens, bad camera exposure, bad camera white balance, bad camera settings, bad camera angle, bad camera lens, bad camera focus, bad camera exposure, bad camera framing, bad camera cropping, bad camera angle, bad camera perspective, bad camera white balance, bad camera settings, bad camera lighting, bad camera shadows, bad camera highlights, bad camera midtones, bad camera saturation, bad camera hue, bad camera brightness, bad camera contrast, bad camera colors, bad camera composition, bad camera shading, bad camera lighting, bad camera focus, bad camera framing, bad camera cropping, bad camera angle, bad camera perspective, bad camera lens, bad camera exposure, bad camera white balance, bad camera settings, bad camera angle, bad camera lens, bad camera focus, bad camera exposure, bad camera framing, bad camera cropping, bad camera angle, bad camera perspective, bad camera white balance, bad camera settings, bad camera lighting, bad camera shadows, bad camera highlights"

    # Generate image with improved parameters
    image = pipe(
        prompt + ", high quality, detailed, sharp focus, professional, accurate, clear, realistic, high resolution, high definition, high res, high def",
        negative_prompt=negative_prompt,
        num_inference_steps=steps,
        guidance_scale=guidance,
        width=768,
        height=768
    ).images[0]
    return image
    return image

gr.Interface(
    fn=generate_image,
    inputs=[
        gr.Textbox(label="Enter Prompt"),
        gr.Slider(20, 50, value=30, step=1, label="Inference Steps"),  # Increased default steps
        gr.Slider(1, 20, value=7.5, step=0.5, label="Guidance Scale")  # Wider range, better default
    ],
    outputs="image"
).launch(share=True, debug=True)